In [49]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# function which returns the number of occurnces of each element of an array
def count_occurrences(array):
    
    unique_elements, counts = np.unique(array, return_counts=True)
    
    occurrences = dict(zip(unique_elements, counts))
    
    return occurrences

# function which returns the local maximums positions and their time value
def local_max(array_x, array_y, fraction):
    
    max_pos = {}
    
    delta = int(len(array_y) / fraction)
    i = 0
    j = 1
    while i < len(array_y):
        
        i_min = i - delta
        i_max = i + delta
        
        if (i_min < 0):    
            i_min = 0
        if (i_max > len(array_y)):
            i_max = len(array_y)
        
        if (array_y[i] == max(array_y[i_min:i_max])):
            
            max_pos[j] = [array_x[i], array_y[i]]
            j += 1
            
            i = i_max - 1
        
        i += 1
    
    return max_pos

# funtion which returns the matrix of all the possible periods of oscillations
def periods(maxima):
    
    p_osc = {}
    
    for key_1 in maxima:
        
        for key_2 in maxima:
        
            if (key_2 > key_1):
                
                p_osc[(key_1, key_2)] = ((maxima[key_2][0] - maxima[key_1][0]) / (key_2 - key_1))
    
    return p_osc

# functions which calculates the mean of the elements of a dictionary
def dic_mean(dic):
    
    n = 0
    sum = 0
    
    for key in dic:
        
        sum += dic[key]
        n += 1
    
    return (sum / n)

# function which returns the highest n values of an array
def find_highest(arr, n):
    
    arr = np.array(arr)
    
    sorted_indices = np.argpartition(arr, -n)[-n:]
    
    highest_values = arr[sorted_indices]
    
    return highest_values

# function which finds index of first element of array less then given value
def find_first_less(array, value):
    
    left = 0
    right = len(array) - 1
    index = -1

    while left <= right:
        
        mid = (left + right) // 2

        if array[mid] < value:
            
            index = mid
            right = mid - 1
        
        else:
            
            left = mid + 1

    return index

# find index of an element of an array
def first_index(array, value):
    
    i = 0
    
    while (i < len(array)):
        
        if (array[i] == value):
            
            return i
        
        i += 1
    
    return -1



# analysis funtion

def analysis(path_equilibrio, path_oscillazioni, int_configurazione, plot_color, max_color, n_bins):

    # import csv files for equilibrium position as DataFrames
    equilibrio = pd.read_csv(path_equilibrio, sep=';')
    # import csv files for oscillating positions as DataFrames
    oscillazione = pd.read_csv(path_oscillazioni, sep=';')

    # convert DatFrames of equilibrium positions to numpy arrays
    eq_time = equilibrio['time'].to_numpy()
    eq_pos = equilibrio['position'].to_numpy()

    # plot equilibrium positions
    plt.scatter(eq_time, eq_pos, marker='.', s=1, c=plot_color)
    plt.grid()
    plt.xlabel('t [s]')
    plt.ylabel('x [m]')
    plt.title('Equilibrium positions measured by photogate for configuration '+str(int_configurazione))
    plt.savefig("analisi/smorzamento/plot_equilibrio_"+str(int_configurazione)+".png", dpi=1200)
    plt.close()

    # plot histogran of equilibrium positions
    fig, axs= plt.subplots(1, 1, tight_layout = True)
    axs.hist(eq_pos, bins = 200, color=plot_color)
    plt.grid()
    plt.xlabel('x [m]')
    plt.ylabel('counts')
    plt.title('Histogram of equilibrium positions for configuration '+str(int_configurazione))
    plt.savefig("analisi/smorzamento/plot_equilibrio_"+str(int_configurazione)+"_hist.png", dpi=1200)
    plt.close()

    # print number of equilibrium positions
    with open('analisi/smorzamento/output_equilibrio_'+str(int_configurazione)+'_hist.log','w') as textfile:
        print("Configuration "+str(int_configurazione)+": ", "\n", file=textfile)
        eq_pos_dic = count_occurrences(eq_pos)
        for pos in eq_pos_dic:
            print("x = ", pos, "\t n: ", eq_pos_dic[pos], file=textfile)

    # convert DatFrames of oscillating positions to numpy arrays
    osc_time = oscillazione['time'].to_numpy()
    osc_pos = oscillazione['position'].to_numpy()

    # print maximum positions
    with open('analisi/smorzamento/output_oscillazioni_'+str(int_configurazione)+'_max.log', 'w') as textfile:
        print("Configuration "+str(int_configurazione)+": ", "\n", file=textfile)
        osc_max_dic = local_max(osc_time, osc_pos, 200)

        for key in osc_max_dic:
            print(key, "\t x: ", osc_max_dic[key][1], "\t t: ", osc_max_dic[key][0], file=textfile)

    # evaluate period of oscillations
    with open('analisi/smorzamento/output_periodi_'+str(int_configurazione)+'.log', 'w') as textfile:
        print("Configuration "+str(int_configurazione)+": ", "\n", file=textfile)
        p_osc = periods(osc_max_dic)

        for i in range(1, len(osc_max_dic)-1):
            for j in range(i+1, len(osc_max_dic)):
                print(i, "-", j, "\t T: ", p_osc[(i, j)], file=textfile)

        T = dic_mean(p_osc)
        print("\n\n\nThe mean period of oscillation is: ", T, file=textfile)

    # plot oscillating positions
    plt.scatter(osc_time, osc_pos, marker='.', s=1, c=plot_color)
    for key in osc_max_dic:
        plt.scatter(osc_max_dic[key][0], osc_max_dic[key][1], marker='.', s=3, c=max_color)
    plt.grid()
    plt.xlabel('t [s]')
    plt.ylabel('x [m]')
    plt.title('Positions measured by photogate for configuration '+str(int_configurazione))
    plt.savefig("analisi/smorzamento/plot_oscillazione_"+str(int_configurazione)+".png", dpi=1200)
    plt.close()
    
    # analysis of oscillating positions and evaluation of the error
    pos_hist, pos_bins = np.histogram(osc_pos, n_bins)
    
    def position(i):
        return ((pos_bins[i]+pos_bins[i+1])/2)
    
    two_high_pos = find_highest(pos_hist, 2)
    i_max_1 = first_index(pos_hist, two_high_pos[0])
    i_max_2 = first_index(pos_hist, two_high_pos[1])
    max_1 = position(i_max_1)
    max_2 = position(i_max_2)
    
    max_mean = (max_1 * pos_hist[i_max_1] + max_2 * pos_hist[i_max_2]) / (pos_hist[i_max_1] + pos_hist[i_max_2])
    
    i_half_1 = find_first_less(pos_hist[0:i_max_1], max_mean)
    i_half_2 = find_first_less(pos_hist[i_max_2:-1], max_mean)
    
    error = ((position(i_half_2) - position(i_half_1)) * sum(pos_hist[i_half_1:i_half_2]) + (position(len(pos_hist) - 2) - position(0)) * (sum(pos_hist[0:i_max_1-1]) + sum(pos_hist[i_max_2-1:-1]))) / sum(pos_hist)
    
    with open('analisi/smorzamento/output_oscillazioni_'+str(int_configurazione)+'_hist.log', 'w') as textfile:
        print("Configuration "+str(int_configurazione)+": ", "\n", file=textfile)
        
        for i in range(len(pos_hist)):           
            print(round(pos_bins[i],6), "-", round(pos_bins[i+1],6), "\tx:", round((pos_bins[i]+pos_bins[i+1])/2,6), "\tn:", pos_hist[i], file=textfile)
        
        print("\n\nThe two peaks of the histogram are at:", file=textfile)
        print(round(pos_bins[i_max_1],6), "-", round(pos_bins[i_max_1+1],6), "\tx:", round(max_1 ,6), "\tn:", pos_hist[i_max_1], file=textfile)
        print(round(pos_bins[i_max_2],6), "-", round(pos_bins[i_max_2+1],6), "\tx:", round(max_2 ,6), "\tn:", pos_hist[i_max_2], file=textfile)
        
        print("\nThe associated error is:", error, file=textfile)
    
    # plot histogran of oscillating positions
    fig, axs= plt.subplots(1, 1, tight_layout = True)
    axs.hist(osc_pos, bins = n_bins, color=plot_color)
    plt.grid()
    plt.xlabel('x [m]')
    plt.ylabel('counts')
    plt.title('Histogram of positions for configuration '+str(int_configurazione))
    plt.savefig("analisi/smorzamento/plot_oscillazione_"+str(int_configurazione)+"_hist.png", dpi=1200)
    plt.close()



analysis('dati/smorzamento/equilibrio_1.csv', 'dati/smorzamento/oscillazione_1.csv', 1, 'blue', 'red', 50)
analysis('dati/smorzamento/equilibrio_2.csv', 'dati/smorzamento/oscillazione_2.csv', 2, 'red', 'blue', 50)
analysis('dati/smorzamento/equilibrio_3.csv', 'dati/smorzamento/oscillazione_3.csv', 3, 'green', 'red', 50)